In [ ]:
import perceval as pcvl
from perceval import BS, catalog, PERM, Circuit, Processor, pdisplay, PS, BasicState
from perceval.rendering.circuit import SymbSkin, DisplayConfig
from exqalibur import FockState

import numpy as np
from numpy import pi
from typing import Optional, Dict, List, Tuple 

DisplayConfig.select_skin(SymbSkin)

## Avant de commencer...

In [ ]:
qubits = {
    "00": BasicState([1, 0, 1, 0]),
}

def measure2p(processor: Processor, input_state: Optional[FockState] = None) -> Dict[FockState, float]:
    """
    Mesure le circuit directement sous forme de Processeur. Cela permet d'automatiser la gestion des "ancilla states" et
    des "heralded gates".

    :param processor:
    :param input_state:
    :return:
    """
    if input_state is None:
        input_state = qubits["00"]

    # On force la règle : la somme des photons par paire de rail doit être égale à 1.
    processor.set_postselection(pcvl.utils.PostSelect("[0,1]==1 & [2,3]==1"))
    processor.min_detected_photons_filter(0)

    # On fait finalement la mesure :
    processor.with_input(input_state)
    measure2p_s = pcvl.algorithm.Sampler(processor)

    return measure2p_s.probs()["results"]

H = BS.H()
RX = BS.Rx
RY = BS.Ry
CNOT = catalog["klm cnot"].build_processor()
NOT = PERM([1, 0])
HP = Circuit(2, "HP") // H // (1, PS(-pi/2))
q = lambda x: [2*x, 2*x+1]
theta = pi/3
gamma = pi/5

# Challenge 4 : De l'inversion

Il est temps de finir cette série épouvantable de challenges. Cette fois, pas de nouvelle notion. Seulement deux circuits, et un objectif : rajouter les deux parties manquantes pour obtenir les résultats attendus. Bonne chance... que le sort vous soit favorable...

In [ ]:
step_one = Circuit(2, "S1") 
p_step_one = Processor("SLOS", 4)
p_step_one.add(q(0), H)
p_step_one.add(q(1), step_one)
p_step_one.add(q(1), RX(-gamma))
p_step_one.add(q(0) + q(1), CNOT)
p_step_one.add([2], PS(theta))
p_step_one.add(q(0) + q(1), CNOT)
p_step_one.add(q(0), H)
p_step_one.add(q(0), RX(theta))
pdisplay(p_step_one)

In [ ]:
print("On attend : {|1,0,0,1>: 1.0}")
measure2p(p_step_one, input_state=qubits["00"])

In [ ]:
step_two = Circuit(2, "S2")
p_step_two = Processor("SLOS", 4)
p_step_two.add(q(0), H)
p_step_two.add(q(1), HP)
p_step_two.add(q(1), RY(theta))
p_step_two.add(q(0) + q(1), CNOT)
p_step_two.add(q(1), RY(-theta))
p_step_two.add(q(0), H)
p_step_two.add(q(1) + q(0), CNOT)
p_step_two.add(q(1), step_two)
pdisplay(p_step_two)

In [ ]:
print("On attend : {|1,0,0,1>: 0.93, |0,1,0,1>: 0.07})")
measure2p(p_step_two, input_state=qubits["00"])

## Récupération du drapeau 

In [ ]:
import requests as rq

def circuit_to_list(circuit: Circuit) -> List[List[Tuple[float, float]]]:
    return [[(x.real, x.imag) for x in l] for l in np.array(circuit.compute_unitary())]


d = {
    "step_one": circuit_to_list(step_one),
    "step_two": circuit_to_list(step_two),
}

URL = "https://perceval.challenges.404ctf.fr"
rq.get(URL + "/healthcheck").json()

In [ ]:
rq.post(URL + "/challenges/4", json=d).json()